Os dados são provenientes do Wyscout e é um dos datasets de futebol abertos mais robustos disponíveis atualmente. Nós temos eventos de todas as partidas de uma temporada completa (2016-2017) das  [cinco principais ligas europeias segundo a UEFA](https://www.uefa.com/memberassociations/uefarankings/country/#/yr/2019) (as primeiras divisões das ligas espanhola; italiana; alemã; francesa e inglesa), do campeonato europeu de seleções de 2016 e da Copa do Mundo da Rússia de 2018.

[Neste artigo](https://www.nature.com/articles/s41597-019-0247-7) há informações detalhadas sobre como se dá a coleta de dados dos jogos de futebol pela Wyscout e a definição das variáveis.

Para cada finalização de todos os campeonatos identificamos:

1) A posição x (horizontal) e y (vertical) do chute no campo;

2) Distância do chute em relação à linha central;

3) Distância do chute (em relação ao gol) em metros;

4) O ângulo do chute em relação à baliza;

5) O campeonato em que o chute foi dado (para análises depois de construir o modelo);

6) Se o chute foi dado a partir de um contra ataque; uma falta; pênalti ou em organização ofensiva da equipe (a equipe adversária estava posicionada defensivamente) e

7) A variável que queremos prever: dado as características acima (tirando o campeonato) foi gol ou não (sua probabilidade)?

[Significados detalhados das tags dos eventos](https://support.wyscout.com/matches-wyid-events#10-available-tag-ids)

*A preparação dos dados foi feita utilizando o Google Colab tendo em vista a capacidade de processamento.

In [1]:
import numpy as np
import pandas as pd
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Chutes da copa do mundo de 2018
with open('/content/drive/MyDrive/events/events_World_Cup.json') as f:
    copa_do_mundo = json.load(f)
    
copa_do_mundo= pd.DataFrame(copa_do_mundo)
copa_do_mundo_chutes= copa_do_mundo[copa_do_mundo['subEventName'].isin(['Shot', 'Penalty', 'Free kick shot'])].assign(campeonato = 'Copa do mundo')

In [ ]:
#Chutes de todas as partidas do campeonato espanhol
with open('/content/drive/MyDrive/events/events_Spain.json') as f:
    espanha = json.load(f)

espanha= pd.DataFrame(espanha)
espanha_chutes= espanha[espanha['subEventName'].isin(['Shot', 'Penalty', 'Free kick shot'])].assign(campeonato = 'Espanhol')

In [ ]:
#Chutes de todas as partidas do campeonato italiano 
with open('/content/drive/MyDrive/events/events_Italy.json') as f:
    italia = json.load(f)

italia= pd.DataFrame(italia)
italia_chutes= italia[italia['subEventName'].isin(['Shot', 'Penalty', 'Free kick shot'])].assign(campeonato = 'Italiano') 

In [ ]:
#Chutes de todas as partidas do campeonato alemão 
with open('/content/drive/MyDrive/events/events_Germany.json') as f:
    alemanha = json.load(f)

alemanha= pd.DataFrame(alemanha)
alemanha_chutes= alemanha[alemanha['subEventName'].isin(['Shot', 'Penalty', 'Free kick shot'])].assign(campeonato = 'Alemao')

In [ ]:
#Chutes de todas as partidas do campeonato francês
with open('/content/drive/MyDrive/events/events_France.json') as f:
    franca = json.load(f)

franca= pd.DataFrame(franca)
franca_chutes= franca[franca['subEventName'].isin(['Shot', 'Penalty', 'Free kick shot'])].assign(campeonato = 'Frances')

In [ ]:
#Chutes de todas as partidas da Eurocopa 2016
with open('/content/drive/MyDrive/events/events_European_Championship.json') as f:
    eurocopa = json.load(f)

eurocopa= pd.DataFrame(eurocopa)
eurocopa_chutes= eurocopa[eurocopa['subEventName'].isin(['Shot', 'Penalty', 'Free kick shot'])].assign(campeonato = 'Eurocopa')


In [ ]:
#Chutes de todas as partidas do campeonato inglês
with open('/content/drive/MyDrive/events/events_England.json') as f:
    inglaterra = json.load(f)

inglaterra= pd.DataFrame(inglaterra)
inglaterra_chutes= inglaterra[inglaterra['subEventName'].isin(['Shot', 'Penalty', 'Free kick shot'])].assign(campeonato = 'Ingles')

In [ ]:
#Juntando dataframes dos chutes de todas as competições
todos_os_chutes= pd.concat([copa_do_mundo_chutes, espanha_chutes, italia_chutes, 
                            alemanha_chutes, franca_chutes, eurocopa_chutes, inglaterra_chutes], axis=0)

In [ ]:
chutes_modelo=pd.DataFrame(columns=['Gol','X','Y', 'Campeonato', 'subEventoNome', 'idTime', 'idJogador'])

for i, chute in todos_os_chutes.iterrows():
  
  cabecalho=0
  for chute_tag in chute['tags']:
    if chute_tag['id']==403:
      cabecalho=1
    #Só inclui o que não é cabeçalho        
    if not(cabecalho):
      #Indicando em qual campeonato o chute foi feito
      chutes_modelo.at[i,'Campeonato']= chute['campeonato']

      #Ids do time e do jogador
      chutes_modelo.at[i,'idTime']= chute['teamId']
      chutes_modelo.at[i,'idJogador']= chute['playerId']
      
      #Incluindo se o chute foi a partir de uma falta, um pênalti, contra ataque ou em organização ofensiva
      if chute_tag['id']==1901:
        chutes_modelo.at[i,'subEventoNome']='counter_attack'
      else:
        chutes_modelo.at[i,'subEventoNome']=chute['subEventName']
      
      #Posição do chute eixo X        
      chutes_modelo.at[i,'X']=100-chute['positions'][0]['x']
      
      #Posição do chute eixo Y
      chutes_modelo.at[i,'Y']=chute['positions'][0]['y']
      
      #Distância do chute da linha central
      chutes_modelo.at[i,'distanciaCentro']=abs(chute['positions'][0]['y']-50)
    
      #Distância do chute (em relação ao gol) em metros
      x=chutes_modelo.at[i,'X']*105/100
      y=chutes_modelo.at[i,'distanciaCentro']*65/100
      chutes_modelo.at[i,'distanciaGol']=np.sqrt(x**2 + y**2)
      
      #Ângulo do chute em radianos.
      angulo = np.arctan(7.32 *x /(x**2 + y**2 - (7.32/2)**2))
      if angulo<0:
        angulo=np.pi+angulo
      chutes_modelo.at[i,'Angulo'] = angulo
    
      #Conferindo se foi gol
      chutes_modelo.at[i,'Gol']=0
      for chute_tag in chute['tags']:
      #As tags nos dizem se o chute foi gol ou não
        if chute_tag['id']==101:
          chutes_modelo.at[i,'Gol']=1

In [ ]:
chutes_modelo.isnull().sum()

In [ ]:
chutes_modelo.head()

In [ ]:
#Salvando dataframe chutes_modelo como arquivo csv
chutes_modelo.to_csv('/content/drive/MyDrive/todos_os_chutes_modelo.csv', index=False)